# Exploratory Data Analysis – IndiaMART Listings

Load scraped data from `output/listings.csv`, summarize, visualize, and note insights.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
path = Path("output/listings.csv")
if not path.exists():
    raise FileNotFoundError("Run scraper.py first to create output/listings.csv")
df = pd.read_csv(path)
df.head(10)

In [ ]:
df.shape, df.dtypes

In [ ]:
df.info()
df.isna().sum()

In [ ]:
df["category"].value_counts()

In [ ]:
# Parse price to numeric for distribution (handles units like "40,000 / Piece")
import re


def parse_price(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    if not s:
        return None

    # Keep the first numeric token; listings often include units like "/ Piece".
    s = s.replace("\u20b9", "").replace("Rs.", "").replace("Rs", "")
    m = re.search(r"(\d[\d,]*\.?\d*)", s)
    if not m:
        return None
    try:
        return float(m.group(1).replace(",", "").strip())
    except ValueError:
        return None


df["price_numeric"] = df["price"].apply(parse_price)
price_valid = df["price_numeric"].dropna()
price_valid.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
df["category"].value_counts().plot(kind="bar", ax=axes[0], color="steelblue")
axes[0].set_title("Listings by Category")
axes[0].set_ylabel("Count")
if len(price_valid) > 0:
    price_valid.clip(upper=price_valid.quantile(0.95)).hist(ax=axes[1], bins=30, color="steelblue", edgecolor="white")
    axes[1].set_title("Price Distribution (capped at 95th percentile)")
    axes[1].set_xlabel("Price (INR)")
plt.tight_layout()
plt.show()

In [ ]:
# Top locations (extract city from location string: first word often city)
df["city"] = df["location"].fillna("").str.split().str[0]
city_counts = df[df["city"] != ""]["city"].value_counts().head(15)
city_counts.plot(kind="barh", figsize=(8, 5), color="teal", alpha=0.8)
plt.title("Top cities by listing count")
plt.xlabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Frequent keywords and top product-type terms (from titles)
import re
from collections import Counter
stop = {"the", "and", "for", "with", "for", "machine", "machinery", "electronic", "electronics", "gadget", "gadgets"}
words = []
for t in df["title"].dropna().astype(str):
    words.extend(re.findall(r"[a-zA-Z0-9]{3,}", t.lower()))
word_counts = Counter(w for w in words if w not in stop)
top_keywords = word_counts.most_common(15)
kw_df = pd.DataFrame(top_keywords, columns=["keyword", "count"])
kw_df.set_index("keyword")["count"].plot(kind="barh", figsize=(8, 5), color="steelblue", alpha=0.8)
plt.title("Frequent keywords in product titles (top 15)")
plt.xlabel("Count")
plt.tight_layout()
plt.show()
print("Top product-type terms:", [w for w, _ in top_keywords[:10]])

In [ ]:
# Supplier-pattern analysis: top suppliers and concentration
supplier_counts = df[df["supplier"].notna() & (df["supplier"].str.strip() != "")]["supplier"].value_counts()
print("Top 10 suppliers by listing count:")
print(supplier_counts.head(10))
top5 = supplier_counts.head(5).sum()
total_supplier = supplier_counts.sum()
pct_top5 = 100 * top5 / total_supplier if total_supplier else 0
print(f"\nConcentration: top 5 suppliers account for {pct_top5:.1f}% of listings with a supplier name.")
supplier_counts.head(10).plot(kind="barh", figsize=(8, 4), color="teal", alpha=0.8)
plt.title("Top 10 suppliers by listing count")
plt.xlabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Data quality and anomalies: missingness, outliers, inconsistent locations
print("=== Missingness by field (NaN + empty strings) ===")


def missing_mask(s):
    if s.dtype == "O":
        return s.isna() | (s.astype(str).str.strip() == "")
    return s.isna()


missing = {col: int(missing_mask(df[col]).sum()) for col in df.columns}
missing_pct = {col: round(100 * missing[col] / len(df), 1) for col in df.columns}
missing_df = (
    pd.DataFrame({"missing": missing, "pct": missing_pct})
    .sort_values("missing", ascending=False)
)
print(missing_df)

print()
print("=== Missingness by category (NaN + empty strings) ===")
for col in ["price", "supplier", "location"]:
    by_cat = df.groupby("category").apply(lambda g: int(missing_mask(g[col]).sum()))
    print(col, "missing by category:", by_cat.to_dict())

print()
print("=== Price outliers (IQR) ===")
if len(price_valid) > 0:
    q1, q3 = price_valid.quantile(0.25), price_valid.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outliers = price_valid[(price_valid < low) | (price_valid > high)]
    print(f"Bounds: [{low:.0f}, {high:.0f}]. Outlier count: {len(outliers)}")
    if len(outliers) > 0:
        print("Sample outlier prices:", outliers.head(5).tolist())

print()
print("=== Location completeness by category ===")
df["has_location"] = df["location"].notna() & (df["location"].str.strip() != "")
print(df.groupby("category")["has_location"].agg(["sum", "count"]).assign(pct=lambda x: 100 * x["sum"] / x["count"]))

In [ ]:
# Sample of titles for keyword sense
df["title"].dropna().head(20).tolist()

## Insights and hypotheses

- **Counts:** Total listings and split by category (Industrial Machinery vs Electronics).
- **Prices:** Most listings have price in INR; distribution is right-skewed; many high-value machinery items.
- **Frequent keywords / product types:** The keyword chart and top terms show dominant product-type language (e.g. machine, automatic, making, portable) and category-specific terms; use these to label “top product types” or refine categories.
- **Supplier patterns:** Top suppliers by count and concentration (e.g. % from top 5) indicate whether the market is fragmented or a few sellers dominate; compare across categories.
- **Regional patterns:** Supplier concentration in cities like Mumbai, Delhi, Ahmedabad, Chennai suggests industrial hubs; city bar chart and location completeness by category summarize this.
- **Anomalies and data quality:** Missingness by field and by category highlights incomplete listings (e.g. many rows with empty supplier/location). IQR-based price outliers flag very low or very high values. Location completeness by category shows where scraping or listing quality drops. Use these to document gaps and inconsistencies in the scraped data.
- **Hypothesis:** Industrial Machinery has higher average price than Electronics; major metros dominate supplier locations.